In [ ]:
import socket
import base64
import hashlib
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.fernet import Fernet

# Create a socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Bind the socket to a specific address and port
server_address = ('localhost', 12341)
server_socket.bind(server_address)

# Listen for incoming connections
server_socket.listen(1)
print("Server is listening on {}:{}".format(*server_address))

# Define hardcoded public keys for the server and client
server_public_key = 5  # Replace with a real public key
client_public_key = 7  # Replace with a real public key

while True:
    print("Waiting for a connection...")

    # Accept a connection
    client_socket, client_address = server_socket.accept()
    print("Accepted connection from {}:{}".format(*client_address))
    
    # Receive the client's public key
    client_public_key_received = int(client_socket.recv(1024).decode())
    
    # Send the server's public key to the client
    client_socket.send(str(server_public_key).encode())

    # Calculate the shared secret
    shared_secret = (client_public_key_received * server_public_key) % 23  # Replace 23 with your prime number

    if shared_secret == 12:  # Replace 12 with the expected shared secret
        print("Key exchange successful. Secrets match.")
        print("Receiving and decrypting the message...")

        # Derive an encryption key from the shared secret using PBKDF2
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            iterations=100000,
            salt=b'',
            length=32,
        )
        encryption_key = base64.urlsafe_b64encode(kdf.derive(shared_secret.to_bytes(32, byteorder='big')))
        
        # Receive and decrypt the message
        encrypted_message = client_socket.recv(1024)
        print("Received Encrypted Message:", encrypted_message)
        cipher = Fernet(encryption_key)
        message_with_hash = cipher.decrypt(encrypted_message)
        
        # Separate the message and hash
        message = message_with_hash[:-32]  # Assuming SHA-256 hash size is 32 bytes
        received_hash = message_with_hash[-32:]
        print("Received Hash:", received_hash)
        # Calculate the SHA-256 hash of the received message
        calculated_hash = hashlib.sha256(message).digest()
        print("Calculated Hash:", calculated_hash)
        
        # Compare the received hash with the calculated hash for integrity verification
        if received_hash == calculated_hash:
            print("Message integrity verified. Received Message:", message.decode())
        else:
            print("Message integrity check failed.")
    else:
        print("Key exchange failed. Secrets do not match.")

    # Close the client socket
    client_socket.close()


Server is listening on localhost:12341
Waiting for a connection...
